In [7]:
import numpy as np

P = np.array([
    [0.65, 0.28, 0.07],
    [0.15, 0.67, 0.18],
    [0.12, 0.36, 0.52]
])

Pn = np.linalg.matrix_power(P, 100)
print(Pn)

pi = Pn[0]
print(pi)

for i in range(0, 3):
    for j in range(0, 3):
        print(pi[i]*P[i][j])
        print(pi[j]*P[j][i])
        print()

[[0.28650138 0.48852158 0.22497704]
 [0.28650138 0.48852158 0.22497704]
 [0.28650138 0.48852158 0.22497704]]
[0.28650138 0.48852158 0.22497704]
0.18622589531680528
0.18622589531680528

0.08022038567493152
0.07327823691460089

0.02005509641873288
0.026997245179063482

0.07327823691460089
0.08022038567493152

0.3273094582185507
0.3273094582185507

0.08793388429752107
0.08099173553719044

0.026997245179063482
0.02005509641873288

0.08099173553719044
0.08793388429752107

0.11698806244260843
0.11698806244260843



In [1]:
pi = [0.286, 0.489, 0.225]
x = sum(it*0.489 for it in pi)
print(x)

0.48899999999999993
